In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from tqdm import tqdm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [26]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']
# df = (df - df.mean())/(df.max() - df.min())
# df0 = df.copy()
# print(df.shape)
# print(target)
df.head()

# train/test split
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.33, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


# Обучаем классификаторы и строим матрицу

In [52]:
from sklearn.linear_model import LogisticRegression

l = np.unique(target).size
N = 50 # кол-во дихотомий
code_matrix = functions.make_random_dichs(l, N)

dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                         X_test, y_test, LogisticRegression)

Training dich classifiers: 100%|██████████| 50/50 [00:01<00:00, 39.08it/s]


In [58]:
attempts_data = []
N_attempts = 10
N = 30 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        for weight_type in wtypes:
            preds = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

100%|██████████| 10/10 [01:00<00:00,  6.04s/it]30 [00:00<00:00, 42.60it/s]


In [59]:
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}_w{}'.format(i1, i2) for i1 in wtypes for i2 in wtypes]
df_attempts

,sNone_wNone,sNone_waccuracy,sNone_wf1,sNone_wconfusion_list,saccuracy_wNone,saccuracy_waccuracy,saccuracy_wf1,saccuracy_wconfusion_list,sf1_wNone,sf1_waccuracy,sf1_wf1,sf1_wconfusion_list,sconfusion_list_wNone,sconfusion_list_waccuracy,sconfusion_list_wf1,sconfusion_list_wconfusion_list
0,0.900463,0.898148,0.898148,0.902778,0.902778,0.902778,0.902778,0.902778,0.902778,0.902778,0.902778,0.902778,0.912037,0.840278,0.840278,0.840278
1,0.907407,0.907407,0.905093,0.909722,0.907407,0.907407,0.907407,0.907407,0.905093,0.905093,0.905093,0.905093,0.909722,0.868056,0.868056,0.868056
2,0.893519,0.900463,0.895833,0.900463,0.893519,0.893519,0.893519,0.893519,0.893519,0.893519,0.893519,0.893519,0.893519,0.835648,0.835648,0.835648
3,0.895833,0.888889,0.900463,0.895833,0.905093,0.905093,0.905093,0.905093,0.905093,0.905093,0.905093,0.905093,0.895833,0.870370,0.870370,0.870370
4,0.893519,0.891204,0.891204,0.893519,0.895833,0.895833,0.895833,0.895833,0.898148,0.898148,0.898148,0.898148,0.905093,0.826389,0.826389,0.826389
5,0.881944,0.886574,0.886574,0.884259,0.893519,0.893519,0.893519,0.893519,0.891204,0.891204,0.891204,0.891204,0.891204,0.798611,0.798611,0.798611
6,0.861111,0.872685,0.868056,0.868056,0.872685,0.872685,0.872685,0.872685,0.870370,0.870370,0.870370,0.870370,0.886574,0.740741,0.740741,0.740741
7,0.893519,0.900463,0.891204,0.902778,0.898148,0.898148,0.898148,0.898148,0.900463,0.900463,0.900463,0.900463,0.902778,0.805556,0.805556,0.805556
8,0.893519,0.898148,0.893519,0.900463,0.898148,0.898148,0.898148,0.898148,0.893519,0.893519,0.893519,0.893519,0.893519,0.831019,0.831019,0.831019
9,0.902778,0.902778,0.900463,0.905093,0.905093,0.905093,0.905093,0.905093,0.902778,0.902778,0.902778,0.902778,0.902778,0.796296,0.796296,0.796296


In [60]:
df_attempts.describe()

,sNone_wNone,sNone_waccuracy,sNone_wf1,sNone_wconfusion_list,saccuracy_wNone,saccuracy_waccuracy,saccuracy_wf1,saccuracy_wconfusion_list,sf1_wNone,sf1_waccuracy,sf1_wf1,sf1_wconfusion_list,sconfusion_list_wNone,sconfusion_list_waccuracy,sconfusion_list_wf1,sconfusion_list_wconfusion_list
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.892361,0.894676,0.893056,0.896296,0.897222,0.897222,0.897222,0.897222,0.896296,0.896296,0.896296,0.896296,0.899306,0.821296,0.821296,0.821296
std,0.012923,0.010075,0.010341,0.012141,0.009953,0.009953,0.009953,0.009953,0.010398,0.010398,0.010398,0.010398,0.008400,0.038236,0.038236,0.038236
min,0.861111,0.872685,0.868056,0.868056,0.872685,0.872685,0.872685,0.872685,0.870370,0.870370,0.870370,0.870370,0.886574,0.740741,0.740741,0.740741
25%,0.893519,0.889468,0.891204,0.894097,0.894097,0.894097,0.894097,0.894097,0.893519,0.893519,0.893519,0.893519,0.893519,0.800347,0.800347,0.800347
50%,0.893519,0.898148,0.894676,0.900463,0.898148,0.898148,0.898148,0.898148,0.899306,0.899306,0.899306,0.899306,0.899306,0.828704,0.828704,0.828704
75%,0.899306,0.900463,0.899884,0.902778,0.904514,0.904514,0.904514,0.904514,0.902778,0.902778,0.902778,0.902778,0.904514,0.839120,0.839120,0.839120
max,0.907407,0.907407,0.905093,0.909722,0.907407,0.907407,0.907407,0.907407,0.905093,0.905093,0.905093,0.905093,0.912037,0.870370,0.870370,0.870370


In [ ]:
df_attempts


# Конструируем модельную задачу

In [ ]:
np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, num_clusters-1, num_clusters):
    for j in np.linspace(0, num_clusters-1, num_clusters):
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
X = np.vstack(X)
scatter(X[:,0], X[:,1], )

In [ ]:
# объединяем классы
classes_in_cluster = 5

order = np.array(range(num_clusters**2))
np.random.shuffle(order)
splits = np.split(order, 5)
for i, split in enumerate(splits):
    for item in split:
        y[item*cluster_objects:(item+1)*cluster_objects] = i

In [ ]:
print(y)
plot(y)

In [ ]:
figsize(8, 8)
scatter(X[:,0], X[:,1], c=y, )

In [ ]:
# модельная задача на 12 классов
colors = []
cs = []

# np.random.seed(100)
cluster_objects = 100
num_clusters = 5

mean = np.zeros(2)
cov = np.eye(2) * 0.07
X = []

y = np.zeros(num_clusters**2*cluster_objects)

for i in np.linspace(0, 4-1, 4):
    for j in np.linspace(0, 5-1, 5):
        c = np.random.randint(0, 12)
        mean = np.array([i, j])
        X_cluster = np.random.multivariate_normal(mean, cov, cluster_objects)
        X.append(X_cluster)
        colors += [c]*cluster_objects
        cs.append(c)
X = np.vstack(X)
print(cs)
scatter(X[:,0], X[:,1], c=colors)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import jaccard_similarity_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_mldata

# Load a multi-label dataset
yeast = fetch_mldata('yeast')
X = yeast['data']
Y = yeast['target'].transpose().toarray()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2,
                                                    random_state=0)

In [ ]:
yeast['target']

In [ ]:
df = pd.read_csv('../yeast.data.txt', sep=';', header=None)
X = df.values[:,1:-1]
y = pd.factorize(df[9])[0]
y

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(X)

In [ ]:
scatter(tsne_results[:,0], tsne_results[:,1], c=y)
